In [3]:
# Import de Librerías
# pip3 install unidecode

import pandas as pd
from unidecode import unidecode

In [4]:
# Leer los CSV de datos históricos
# reportes21 = Reportes 2021 - 2022
# reportes24 = Reportes 2023 - 2024

reportes21 = pd.read_csv('./analisis_reportes_agua/src/historicos_reportes_agua_cdmx/reportes_agua_hist.csv')
reportes24 = pd.read_csv('./analisis_reportes_agua/src/historicos_reportes_agua_cdmx/reportes_agua_2024_01.csv')

In [6]:
# Quitar columnas innecesarias
limpieza21 = reportes21.drop(columns=['folio', 'quien_atiende', 'codigo_postal', 'colonia_registro_sacmex', 'colonia_datos_abiertos'])
limpieza24 = reportes24.drop(columns=['folio_incidente', 'id_reporte', 'hora_reporte', 'medio_recepcion', 'colonia_catalogo', 'fecha_registro_incidente', 'clasificacion'])

# Cambiar el nombre de las columnas
limpieza24 = limpieza24.rename(columns={'fecha_reporte': 'fecha'})
limpieza24 = limpieza24.rename(columns={'reporte': 'tipo_de_falla'})
limpieza24 = limpieza24.rename(columns={'alcaldia_catalogo': 'alcaldia'})

# Darles el mismo orden a ambos Dataframes
ordenNuevo = ['fecha', 'tipo_de_falla', 'alcaldia', 'longitud', 'latitud']
limpieza21 = limpieza21[ordenNuevo]

# Limpiar filas con datos vacíos en alguna de las columnas ['alcaldia', 'longitud', 'latitud', 'tipo_de_falla']
limpieza21 = limpieza21.dropna(subset=['alcaldia', 'longitud', 'latitud', 'tipo_de_falla'])
limpieza24 = limpieza24.dropna(subset=['alcaldia', 'longitud', 'latitud', 'tipo_de_falla'])

# Cambiar formato a Mayúsculas en columna 'alcaldia'
limpieza24['alcaldia'] = limpieza24['alcaldia'].str.upper()

# Quitar acentos de la columna 'alcaldia'
limpieza24['alcaldia'] = limpieza24['alcaldia'].apply(unidecode)

# Función para insertar un punto en la posición 3 de una cadena
def insertar_punto_longitud(cadena):
    return cadena[:3] + '.' + cadena[3:]

# Función para insertar un punto en la posición 2 de una cadena
def insertar_punto_latitud(cadena):
    return cadena[:2] + '.' + cadena[2:]

# Eliminar el carácter '.' de la columna 'longitud' y 'latitud
limpieza24['longitud'] = limpieza24['longitud'].astype(str).str.replace('.', '', regex=False)
limpieza24['latitud']  = limpieza24['latitud'].astype(str).str.replace('.', '', regex=False)

# Eliminar el carácter '.' de la columna 'longitud' y 'latitud
limpieza21['longitud'] = limpieza21['longitud'].astype(str).str.replace('.', '', regex=False)
limpieza21['latitud']  = limpieza21['latitud'].astype(str).str.replace('.', '', regex=False)

# Insertar a todos los valores el carácter '.'
limpieza24['longitud'] = limpieza24['longitud'].apply(insertar_punto_longitud)
limpieza24['latitud']  = limpieza24['latitud'].apply(insertar_punto_latitud)

# Insertar a todos los valores el carácter '.'
limpieza21['longitud'] = limpieza21['longitud'].apply(insertar_punto_longitud)
limpieza21['latitud']  = limpieza21['latitud'].apply(insertar_punto_latitud)

# Fusionar los dos CSV
historico = pd.concat([limpieza21, limpieza24], ignore_index=True)

# Guardar CSV limpios
limpieza21.to_csv('./analisis_reportes_agua/src/csv/limpieza21_22.csv')
limpieza24.to_csv('./analisis_reportes_agua/src/csv/limpieza23_24.csv')
historico.to_csv('./analisis_reportes_agua/src/csv/historico_completo.csv')
